# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
3C OK
PASS OK
4C ? NN-value: PASS 0.972
4H OK
PASS OK
PASS OK
PASS ? NN-value: 5C 0.436
analyzing the play
CA


Loaded lib dds.dll


C8 OK
C3 OK
C5 OK
S8 OK
ST OK
SQ OK
SA OK
DK ? losing: 0.40
DT OK
D6 OK
D3 OK
D9 ?? losing: 1.00
D7 OK
DA OK
D8 OK
DJ OK
DQ OK
HJ OK
S7 OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '3C',
 'candidates': [{'call': '3C', 'insta_score': 0.959}],
 'samples': [],
 'hcp': -1,
 'shape': -1,
 'who': 'Analysis'}

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

{'card': 'CA',
 'candidates': [{'card': 'DT',
   'insta_score': 0.032,
   'expected_tricks_sd': 11.71,
   'p_make_contract': 0.08},
  {'card': 'CA',
   'insta_score': 0.955,
   'expected_tricks_sd': 11.55,
   'p_make_contract': 0.05}],
 'samples': ['KQJxx.xxx.KJxx.8 T9xx.T.9xxx.Txxx Ax.AKQJ9xx.AQ8.Q 0.34878',
  'KQT9.QTxx.Q8x.Q8 xxxx.9.J9xxx.xxx AJx.AKJxxx.AKx.T 0.31813',
  'AQxx.Jxx.QJ9x.Qx xxx.Tx.8xxxx.Txx KJT9.AKQ9xx.AK.8 0.31518',
  'KQJxx.KJ9.J8xxx. Txx.Txx.xx.T8xxx A9x.AQxxx.AKQ9.Q 0.31224',
  'QJ9xx.9xxx.K9xx. Txxx.Tx.8x.T8xxx AK.AKQJx.AQJxx.Q 0.30247',
  'QTxx.9xxx.K9x.xx 9xx.x.AQJxxx.T8x AKJx.AKQJTx.8x.Q 0.29973',
  'QJTx.Qxxx.Qxx.QT xxx.x.AK9xxx.xxx AK9x.AKJT9x.J8.8 0.29953',
  'AJ9xx.ATx.Qxx.T8 Tx.9xx.98xxx.xxx KQxx.KQJxx.AKJ.Q 0.29819',
  'AQJTx.JTxx.Kxx.x xx.9.AQJ8xx.T8xx K9xx.AKQxxx.9x.Q 0.29071',
  'QJTx.JTx.J9xx.T8 9xx.x.AQ8xxx.Qxx AKxx.AKQ9xxx.K.x 0.29010'],
 'hcp': -1,
 'shape': -1}

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'H9',
 'candidates': [{'card': 'H9',
   'insta_score': 1.0,
   'expected_tricks_dd': 2.3,
   'p_make_contract': 0.43,
   'expected_score_dd': -224},
  {'card': 'HK',
   'insta_score': 0.0,
   'expected_tricks_dd': 0.72,
   'p_make_contract': 0.0,
   'expected_score_dd': -458}],
 'samples': ['8xx.x.T9.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxxx.Kx.x 0.779',
  '8xx.x.9x.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxxx.KT.x 0.775',
  '8xx.x.Tx.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxxx.K9.x 0.772',
  '8x.x.T9x.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axx.AJ8xxxxx.K.x 0.692',
  '8xx.xx.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxx.K9x.x 0.511',
  '8xx.xx.9.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxx.KTx.x 0.509',
  '8xx.xx.x.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxx.KT9.x 0.505',
  '8xx.8x.9.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJxxxxx.KTx.x 0.503',
  '8xx.8x.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJxxxxx.K9x.

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)

In [12]:
card_by_card = CardByCard(*board, models, sampler, False)

In [13]:
card_by_card.analyze()

analyzing the bidding
1C OK
2H OK
X OK
4H ? NN-value: PASS 0.324
4S ? NN-value: 5C 0.049
PASS OK
PASS OK
PASS OK
analyzing the play
DA
D3 OK
D5 OK
D6 OK
C8 OK
C3 OK
CJ OK
CA OK
S4 ?? losing: 1.75
S5 OK
S8 OK
SJ OK
H5 OK
HJ OK
HQ OK
HA OK
S2 OK
SA OK
ST OK
S3 OK
H2 OK
SK OK
H4 OK
H3 OK
D8 OK
D2 OK
DJ OK
S6 OK
SQ OK
C6 ?? losing: 1.00
H7 OK
S7 OK
H8 OK
C7 OK
HK OK
S9 OK
C5 OK
C2 OK
CT OK
HT OK
CK OK
H9 OK
D7 OK
C4 OK
DQ OK
D4 OK
DK OK
H6 OK


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [14]:
card_by_card.cards['S4'].to_dict()

{'card': 'CK',
 'candidates': [{'card': 'CK',
   'insta_score': 0.107,
   'expected_tricks_dd': 6.85,
   'p_make_contract': 0.0,
   'expected_score_dd': -108},
  {'card': 'DQ',
   'insta_score': 0.113,
   'expected_tricks_dd': 6.85,
   'p_make_contract': 0.0,
   'expected_score_dd': -108},
  {'card': 'D8',
   'insta_score': 0.046,
   'expected_tricks_dd': 6.85,
   'p_make_contract': 0.0,
   'expected_score_dd': -108},
  {'card': 'HJ',
   'insta_score': 0.325,
   'expected_tricks_dd': 6.1,
   'p_make_contract': 0.0,
   'expected_score_dd': -145},
  {'card': 'C7',
   'insta_score': 0.222,
   'expected_tricks_dd': 5.45,
   'p_make_contract': 0.0,
   'expected_score_dd': -178},
  {'card': 'C6',
   'insta_score': 0.222,
   'expected_tricks_dd': 5.45,
   'p_make_contract': 0.0,
   'expected_score_dd': -178},
  {'card': 'C9',
   'insta_score': 0.021,
   'expected_tricks_dd': 5.45,
   'p_make_contract': 0.0,
   'expected_score_dd': -178},
  {'card': 'CT',
   'insta_score': 0.012,
   'expected_

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [15]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'candidates': [{'card': 'C8',
   'insta_score': 0.085,
   'expected_tricks_sd': 12.04,
   'p_make_contract': 0.01},
  {'card': 'HK',
   'insta_score': 0.837,
   'expected_tricks_sd': 11.96,
   'p_make_contract': 0.01}],
 'samples': ['JT98x.x.KJx.AQxx x.AJ8xxx.Q8xxx.x AKQxx..9.KJT9xxx 0.50374',
  'QJT9xx.x.KQJ.Kxx 8.AJ8xxx.98x.QJT AKxx..xxx.A9xxxx 0.50338',
  'AJT8x.x.KJ8.KJxx x.AJ8xxx.Q9xxx.x KQ9xx..x.AQT9xxx 0.47506',
  'AKQxx.xx.QJ8x.xx T.AJ8xx.K9x.T9xx J98xx..xx.AKQJxx 0.46506',
  'ATxx..KJ98xx.Axx 98.AJ8xxxx.Q.T9x KQJxx..xx.KQJxxx 0.46324',
  'KJT8x.8x.KQ.QT9x x.AJxxx.J98xxx.x AQ9xx..x.AKJxxxx 0.46099',
  'AQJ9xx.J8.Kxx.Jx .Axxxx.Q98x.QT9x KT8xx..Jx.AKxxxx 0.45736',
  'AQxxx.A8.Qxx.Txx 9.Jxxxx.KJ8x.QJ9 KJT8x..9x.AKxxxx 0.45580',
  'AKJT8x.x.Jxx.QTx 9.AJ8xx.Q98x.J9x Qxxx.x.Kx.AKxxxx 0.45523',
  'AQJTxx..K8xx.Jxx x.AJxxx.QJxx.9xx K98x.8x.9.AKQTxx 0.44782'],
 'hcp': -1,
 'shape': -1}